# asdfasd

&nbsp;

## Overview

asdfasd

&nbsp;

## Conclusion

asdfasd

&nbsp;

##### References:
 - [text](link)

-----------------------------------

In [ ]:
## imports & configuration

# standard imports

# custom imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile
import simpleaudio as sa
import sympy as sp
import tqdm.notebook

# inline plots
%config InlineBackend.figure_formats = ['svg']
# %matplotlib notebook

# jupyter theme
try:
    import jupyterthemes as jt
    jt.jtplot.style()
except ImportError:
    pass

-----------------------------------

In [ ]:
## methods to work with audio

# load wav file as float array
def load_wav(path):
    # retrieve sample rate and raw data
    sample_rate, data = scipy.io.wavfile.read(path)

    # compute bit depth
    bit_depth = data.ravel()[0].data.nbytes*8

    # convert signal to float
    signal = data/(2**(bit_depth-1))

    # convert to 2D array (to support mono/stereo)
    signal = np.atleast_2d(signal.T).T

    # return signal and sample rate
    return signal, sample_rate

# play audio signal
def play(signal, sample_rate=44100, bit_depth=16):
    # convert signal to 2D array
    signal = np.atleast_2d(signal.T).T

    # convert float to int signal
    signal = (signal*(2**(bit_depth-1)-1)).astype(getattr(np, f"int{bit_depth}"))

    # play signal
    sa.play_buffer(signal, signal.shape[1], bit_depth//8, sample_rate).wait_done()

# save signal as wav file
def save_wav(signal, path, sample_rate=44100):
    signal = np.atleast_2d(signal.T).T
    scipy.io.wavfile.write(path, sample_rate, signal)

In [ ]:
# load audio data
signal, sample_rate = load_wav(
#     "/Users/ktabrizi/Documents/Development/sandbox/python/quick_reverb/jason_dont_believe_me.wav"
    "/Users/ktabrizi/Documents/Development/sandbox/python/misc_notebooks/test.wav"
)

-----------------------------------

In [ ]:
# initial mono prototype
def prototype0(signal, f_min=100, f_max=20000, n_f=1000, sample_rate=44100, tail_buffer=2, seed=None):
    # initialize input signal with empty buffer tail
    n = len(signal)+tail_buffer*sample_rate
    in_signal = np.zeros((n,))
    in_signal[:len(signal)] = signal[:, 0]

    # set constants
    dt = 1/sample_rate
    rng = np.random.default_rng(seed=seed)
    k_q, k_s = 100, f_min+(f_max-f_min)*np.sqrt(rng.random(n_f))
    r_q, r_s = -5, 5

    # initialize simulation state
    pos, vel = r_s*np.ones(n_f), np.zeros(n_f)
    pre = np.empty((n, n_f))

    # loop sample-by-sample
    for i in tqdm.notebook.tqdm(range(n)):
        inp = in_signal[i]
        F_q = -k_q*inp #/((pos-r_q)**2)
        F_s = -k_s*(pos-r_s)
        acc = (F_q+F_s)/.0001
        vel += dt*acc
        vel *= 0.9999 # this should be frequency dependent
        pos += dt*vel
        pre[i] = (pos-r_s)

    # return maybe-normalized sum of oscillator values
    out = pre.sum(axis=-1)
    out /= max(1, abs(out).max())
    return out

In [ ]:
# play prototype
play(prototype0(signal[:100000]))

-----------------------------------

In [ ]:
# sympy symbol setup
r0, th0, dy = sp.symbols('r_0 \\theta_0 \Delta\ y')
x0 = r0 * sp.cos(th0)
y0 = r0 * sp.sin(th0)
y1 = y0 + dy

In [ ]:
# analytic expression of change in radius
dr_expr = sp.simplify(sp.sqrt((x0**2 + y1**2))-r0)
sp.symbols('\Delta\ r\ =\ 0') + dr_expr

In [ ]:
# analytic expression of change in theta
tan_th0, tan_th1 = y0/x0, (y0+dy)/x0
dth_expr = sp.trigsimp(sp.arg((tan_th1 - tan_th0)/(1 + tan_th0 * tan_th1)))
sp.symbols('\Delta\ \\theta\ =\ 0') + dth_expr

In [ ]:
# new "integration-free" prototype (not working...)
def _prototype1_(signal, sample_rate, fs, ws, ds):
    # set constants
    dt = 1/sample_rate
    os = 2*np.pi*fs
    cs = (1-ds*dt)
    n_f = len(fs)

    # validate inputs
    assert fs.shape == ws.shape
    assert fs.shape == ds.shape
    assert np.all(cs >= 0) # no negative damping
    assert np.all(cs <= 1) # no amplitude growth

    # initialize containers for simulation state
    t0 = 0 # this should be global time of first input sample 
    amp, toff = np.zeros(n_f), np.zeros(n_f)
    pos = np.empty((len(signal), n_f))

    # loop sample-by-sample
    for i in tqdm.notebook.tqdm(range(len(signal))):
        # pre-compute values
        th0 = os*(t0+i*dt+toff)
        cos_th0 = np.cos(th0)
        sin_th0 = np.sin(th0)
        amp2 = amp**2

        # update amplitudes and offset times
        dv = signal[i]
        toff += np.arctan2(dv*cos_th0, dv*sin_th0+amp)
        amp = np.sqrt(dv**2 + 2*dv*amp*sin_th0 + amp2)

        # compute spring positions
        pos[i] = amp*np.cos(os*(t0+i*dt)+toff)

        # damp position & velocity
        amp *= cs

    # return weighted mean of oscillator values
    return pos.dot(ws)/ws.sum()

# wrapper for pre/post-processing of signal and parameters
def prototype1(
    raw_signal, ds=None, fs=None, ws=None,
    f_min=100, f_max=20000, n_f=1000,
    seed=None, sample_rate=44100, tail_buffer=2
):
    # initialize input signal with empty buffer tail
    n = len(raw_signal)+tail_buffer*sample_rate
    signal = np.zeros((n,))
    signal[:len(raw_signal)] = raw_signal.copy()

    # default frequencies to a random distribution
    if fs is None:
        rng = np.random.default_rng(seed=seed)
        fs = f_min+(f_max-f_min)*rng.random(n_f)

    # set default weights and damping times
    if ws is None:
        ws = np.ones(len(fs))
    if ds is None:
        ds = np.ones(len(fs))*5
    elif isinstance(ds, (float, int)):
        ds = np.ones(len(fs))*ds

    # convert signal to 2d
    signal = np.atleast_2d(signal.T).T

    # retrieve output per channel
    out = np.stack([
        _prototype1_(channel, sample_rate, fs, ws, ds)
        for channel in signal.T
    ]).T.copy()

    # return overload-protection-normalized signal
    return out / max(1, abs(out).max())

In [ ]:
# play(prototype1(signal[:50000, 0], ds=0.))

In [ ]:
# play(prototype1(signal[:100000].T.reshape(2, -1).T))

-----------------------------------

In [ ]:
# original approach but more correct?
def _prototype2_(signal, sample_rate, fs, ws, ds):
    # set constants
    dt = 1/sample_rate
    ks = (2*np.pi*fs/sample_rate)**2
    cs = (1-ds*dt)
    n_f = len(fs)

    # validate inputs
    assert fs.shape == ws.shape
    assert fs.shape == ds.shape
    assert np.all(cs >= 0) # no negative damping
    assert np.all(cs <= 1) # no amplitude growth

    # initialize containers for simulation state
    pos, vel = np.zeros(n_f), np.zeros(n_f)
    positions = np.empty((len(signal), n_f))

    # loop sample-by-sample
    for i in tqdm.notebook.tqdm(range(len(signal))):
        # compute accelerations (mass is 1)
        acc_signal = -1000000*signal[i]
        acc_spring = -1000000*ks*pos
        acc = acc_spring+acc_signal

        # integrate
        vel += dt*acc
#         vel *= cs # this should be frequency independent
        pos += dt*vel

        # write positions
        positions[i] = pos.copy()

    # return weighted mean of oscillator values
    return positions.dot(ws)/ws.sum()

# wrapper for pre/post-processing of signal and parameters
def prototype2(
    raw_signal, ds=None, fs=None, ws=None,
    f_min=1000, f_max=20000, n_f=1000,
    seed=None, sample_rate=44100, tail=2
):
    # initialize input signal with empty buffer tail
    n = len(raw_signal)+tail*sample_rate
    signal = np.zeros((n,))
    signal[:len(raw_signal)] = raw_signal.copy()

    # default frequencies to a random distribution
    if fs is None:
        rng = np.random.default_rng(seed=seed)
        fs = f_min+(f_max-f_min)*rng.random(n_f)

    # set default weights and damping times
    if ws is None:
        ws = np.ones(len(fs))
    if ds is None:
        ds = np.ones(len(fs))*5
    elif isinstance(ds, (float, int)):
        ds = np.ones(len(fs))*ds

    # convert signal to 2d
    signal = np.atleast_2d(signal.T).T

    # retrieve output per channel
    out = np.stack([
        _prototype2_(channel, sample_rate, fs, ws, ds)
        for channel in signal.T
    ]).T.copy()

    # return overload-protection-normalized signal
    return out / abs(out).max()
    return out / max(1, abs(out).max())

In [ ]:
# still not working
plt.plot(signal[:100000, 0])
output = prototype2(signal[:100000, 0], ds=5, f_min=10000, f_max=20000, tail=1)
plt.plot(output)
plt.tight_layout()
play(output)

-----------------------------------

In [ ]:
# general wrapper for pre/post-processing of signal and parameters
def wrapper(
    method, raw_signal, ds=None, fs=None, ws=None,
    f_min=100, f_max=20000, n_f=1000,
    seed=None, sample_rate=44100, tail=2, normalize=False
):
    # convert raw signal to 2d
    raw_signal = np.atleast_2d(raw_signal.T).T

    # initialize input signal with empty buffer tail
    n = len(raw_signal)+tail*sample_rate
    signal = np.zeros((n, raw_signal.shape[1]))
    signal[:len(raw_signal), :] = raw_signal.copy()

    # default frequencies to a random distribution
    if fs is None:
        rng = np.random.default_rng(seed=seed)
#         fs = f_min+(f_max-f_min)*rng.random(n_f) # linear
        fs = np.exp(np.log(f_min)+(np.log(f_max)-np.log(f_min))*rng.random(n_f)) # log

    # set default weights and damping times
    if ws is None:
        ws = np.ones(len(fs))
    if ds is None:
        ds = np.ones(len(fs))*1
    elif isinstance(ds, (float, int)):
        ds = np.ones(len(fs))*ds

    # convert damping times to decay coefficients
    cs = (1-ds/sample_rate)

    # normalize weights
    ws /= ws.sum()

    # validate inputs
    assert fs.shape == ws.shape
    assert fs.shape == ds.shape
    assert np.all(cs >= 0) # no negative damping
    assert np.all(cs <= 1) # no amplitude growth

    # retrieve output per channel
    out = np.stack([
        method(channel, sample_rate, fs, ws, cs)
        for channel in signal.T
    ]).T.copy()

    # return optionally-normalized signal
    if normalize is None:
        return np.clip(out, -1, 1)
    elif normalize is False:
        return out / max(1, abs(out).max())
    elif normalize == "raw":
        return out
    else:
        return out / abs(out).max()

In [ ]:
# fourier fly-wheel idea
def prototype3(signal, sample_rate, fs, ws, cs):
    # set constants
    dt = 1/sample_rate
    os = 2*np.pi*fs*dt
    n = len(signal)
    n_f = len(fs)

    # normalize weights
    ws /= ws.sum()

    # validate inputs
    assert fs.shape == ws.shape
    assert fs.shape == cs.shape
    assert np.all(cs >= 0) # no negative damping
    assert np.all(cs <= 1) # no amplitude growth

    # initialize containers
    pos = np.zeros((n_f,), dtype=np.complex128)
    output = np.zeros((n,))

    # loop through samples
    for i in tqdm.notebook.tqdm(range(n)):
        # unit complex oscillator value
        unit = np.exp(1j*i*os)

        # accumulate scaled complex values
        pos += signal[i]*unit

        # damp cumulant magnitudes
        pos *= cs

        # store oscillator component at cumulant magnitude
        output[i] = (np.abs(pos)*np.real(unit)).dot(ws)

    # return output signal
    return output

In [ ]:
# this works, but it's slow and doesn't sound great
plt.plot(signal[:100000, 0])
output = wrapper(prototype3, signal[:100000, 0], ds=1, n_f=1000, f_min=100, f_max=1000, tail=2, normalize=False)
# output = wrapper(prototype3, signal[:100000, 0], ds=5, n_f=1000, f_min=1000, f_max=20000, tail=2, normalize=None)
plt.plot(output)
plt.tight_layout()
play(output)

-----------------------------------

In [ ]:
signal, sample_rate = load_wav(
    "/Users/ktabrizi/Documents/Development/sandbox/python/quick_reverb/jason_dont_believe_me.wav"
#     "/Users/ktabrizi/Documents/Development/sandbox/python/misc_notebooks/test.wav"
)

In [ ]:
# once again, return to initial approach
def prototype4(signal, sample_rate, fs, ws, cs):
    # TODO: see if better integrator alleviate this
    if np.any(fs > sample_rate/4):
        raise ValueError("Spring frequencies above a fourth of the sample rate aren't supported!")

    # set constants
    n = len(signal)
    n_f = len(fs)
    k_spring = (2*np.pi*fs/sample_rate)**2

    # initialize simulation state
    pos, vel = np.zeros(n_f), np.zeros(n_f)
    output = np.zeros((n,))

    # loop through samples
    for i in tqdm.notebook.tqdm(range(n)):
        acc = signal[i] + (-k_spring*pos) # mass is 1
        vel += acc
        pos += vel
        vel *= cs
        out = pos*np.sqrt(k_spring) # correct for frequency-dependent RMS deviation
        output[i] = out.mean(axis=-1) # average result (volume should be independent of # of frequencies)

    # return maybe-normalized sum of oscillator values
    return output

In [ ]:
# lucky break, sounds like improvement
def prototype5(signal, sample_rate, fs, ws, cs):
    # TODO: see if better integrator alleviate this
    if np.any(fs > sample_rate/4):
        raise ValueError("Spring frequencies above a fourth of the sample rate aren't supported!")

    # set constants
    n = len(signal)
    n_f = len(fs)
    k_spring = (2*np.pi*fs/sample_rate)**2

    # initialize simulation state
    pos, vel = np.zeros(n_f), np.zeros(n_f)
    output = np.zeros((n,))

    # compute diffed signal (TODO: why does this work??)
    diffed = np.zeros_like(signal)
    diffed[:-1] = np.diff(signal)

    # loop through samples
    for i in tqdm.notebook.tqdm(range(n)):
        acc = diffed[i] + (-k_spring*pos) # mass is 1
        vel += acc
        pos += vel
        vel *= cs
        out = pos*np.sqrt(k_spring) # correct for frequency-dependent RMS deviation
        output[i] = out.mean(axis=-1) # average result (volume should be independent of # of frequencies)

    # return maybe-normalized sum of oscillator values
    return output

In [ ]:
# section = signal[:50000]
section = signal[60000:100000]
# section = np.tile(signal[60000:100000], 2)
# section = signal[-200000:-150000]

In [ ]:
# evaluate prototype
# output = wrapper(prototype5, section, ds=5, n_f=1000, f_min= 500, f_max=1000, tail=2, normalize="raw", seed=0)
# output = wrapper(prototype5, section, ds=5, n_f=1000, f_min=2000, f_max=5000, tail=2, normalize="raw", seed=0)
output = wrapper(prototype5, section, ds=5, n_f=1000, f_min=100, f_max=10000, tail=2, normalize="raw", seed=0)

In [ ]:
# play prototype
play(output/abs(output).max())

In [ ]:
# plot prototype
plt.plot(output)
plt.plot(section)
plt.legend(["wet", "dry"])
plt.tight_layout()

-----------------------------------

In [ ]:
# try at various frequency bands
section = signal[60000:100000, :1]
fll, flh, fhl, fhh = 100, 200, 2000, 10000
low  = wrapper(prototype5, section, ds=5, n_f=1000, f_min=fll, f_max=flh, tail=2, normalize=True, seed=0)
high = wrapper(prototype5, section, ds=5, n_f=1000, f_min=fhl, f_max=fhh, tail=2, normalize=True, seed=0)
padded = np.zeros_like(low)
padded[:len(section)] = section

In [ ]:
## plot spectrograms

%config InlineBackend.figure_formats = ['png']
import scipy.signal

plt.figure(figsize=(10, 10))

for i, s in enumerate([padded, low, high]):
    plt.subplot(3, 1, i+1)
    f, t, Sxx = scipy.signal.spectrogram(s.ravel(), sample_rate, nperseg=512)
    plt.pcolormesh(t, f, Sxx, shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.yscale('symlog')
    plt.xlim(0, 1)
    if i == 0:
        plt.axhline(fll, lw=0.5, ls='--')
        plt.axhline(flh, lw=0.5, ls='--')
        plt.axhline(fhl, lw=0.5, ls='--')
        plt.axhline(fhh, lw=0.5, ls='--')
    if i == 1:
        plt.axhline(fll, lw=0.5, ls='--')
        plt.axhline(flh, lw=0.5, ls='--')
    if i == 2:
        plt.axhline(fhl, lw=0.5, ls='--')
        plt.axhline(fhh, lw=0.5, ls='--')

plt.tight_layout()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

In [ ]:
# plot individual reverb sections
plt.figure(figsize=(7, 3))
plt.plot(section)
plt.plot(low)
plt.plot(high)
plt.xlim(11000, 12000)
plt.legend(["raw", "low", "high"])
plt.tight_layout()

-----------------------------------

In [ ]:
signal, sample_rate = load_wav(
    "/Users/ktabrizi/Documents/Development/sandbox/python/quick_reverb/kirby.wav"
)

In [ ]:
section = signal[0:100000, :1]

In [ ]:
play(section)

In [ ]:
#
def prototype6(signal, sample_rate, fs, ws, cs):
    # TODO: see if better integrator alleviate this
    if np.any(fs > sample_rate/4):
        raise ValueError("Spring frequencies above a fourth of the sample rate aren't supported!")

    # set constants
    n = len(signal)
    n_f = len(fs)
    k_spring = (2*np.pi*fs/sample_rate)**2

    # initialize simulation state
    pos, vel = np.zeros(n_f), np.zeros(n_f)
    output = np.zeros((n,))

    # compute diffed signal (TODO: why does this work??)
    diffed = np.zeros_like(signal)
    diffed[:-1] = np.diff(signal)
    cumsum = np.cumsum(diffed)
    cumsum = 0

    # loop through samples
    for i in tqdm.notebook.tqdm(range(n)):
        acc = (-k_spring*pos) # mass is 1
        vel += acc
        pos += diffed[i] + vel
        vel *= cs
        out = (pos)*np.sqrt(k_spring) # correct for frequency-dependent RMS deviation
        output[i] = out.mean(axis=-1) # average result (volume should be independent of # of frequencies)

    # return maybe-normalized sum of oscillator values
    return output

In [ ]:
#
def prototype7(signal, sample_rate, fs, ws, cs):
    # TODO: see if better integrator alleviate this
    if np.any(fs > sample_rate/4):
        raise ValueError("Spring frequencies above a fourth of the sample rate aren't supported!")

    # set constants
    n = len(signal)
    n_f = len(fs)
    k_spring = (2*np.pi*fs/sample_rate)**2

    # initialize simulation state
    pos, vel = np.zeros(n_f), np.zeros(n_f)
    output = np.zeros((n,))

#     # compute diffed signal (TODO: why does this work??)
    diffed = np.zeros_like(signal)
    diffed[:-1] = np.diff(signal)
    cumsum = 0
    prev_sig = 0

    # loop through samples
    for i in tqdm.notebook.tqdm(range(n)):
        acc = (-k_spring*(pos)) # mass is 1
        vel += acc # dt is 1
        pos += vel + signal[i] - prev_sig
        cumsum += signal[i]
        vel *= cs
        out = (pos-signal[i])*np.sqrt(k_spring) # correct for frequency-dependent RMS deviation
        prev_sig = signal[i]
        output[i] = out.mean(axis=-1) # average result (volume should be independent of # of frequencies)

#         out = pos*np.sqrt(k_spring) # correct for frequency-dependent RMS deviation
#         out = (pos-prev_sig)*np.sqrt(k_spring) # correct for frequency-dependent RMS deviation
#         out = (pos-cumsum)*np.sqrt(k_spring) # correct for frequency-dependent RMS deviation
#         out = (pos-(cumsum+signal[i]))*np.sqrt(k_spring) # correct for frequency-dependent RMS deviation

    # return maybe-normalized sum of oscillator values
    return output

In [ ]:
## prototype 5 is well-rounded, works by filtering the low-end
## prototype 7 provides clean wet bottom end
## prototype 8 combines these

In [ ]:
#
def prototype8(signal, sample_rate, fs, ws, cs):
    # TODO: see if better integrator alleviate this
    if np.any(fs > sample_rate/4):
        raise ValueError("Spring frequencies above a fourth of the sample rate aren't supported!")

    # set constants
    n = len(signal)
    n_f = len(fs)
    k_spring = (2*np.pi*fs/sample_rate)**2

    # determine low/high mix coefficients for each frequency
    LOW_LIM = 100 # frequency (hZ) below which only low-end signal is used
    HIGH_LIM = 10000 # frequency (hZ) above which only high-end signal is used
    frac_fs = (np.log(fs)-np.log(LOW_LIM))/(np.log(HIGH_LIM)-np.log(LOW_LIM))
    frac_fs = np.clip(frac_fs, 0., 1.)

    # initialize simulation state
    pos, vel = np.zeros(n_f), np.zeros(n_f)
    output = np.zeros((n,))

#     # compute diffed signal (TODO: why does this work??)
    diffed = np.zeros_like(signal)
    diffed[:-1] = np.diff(signal)
    prev_sig = 0

    # loop through samples
    for i in tqdm.notebook.tqdm(range(n)):
        acc = (-k_spring*(pos)) # mass is 1
        vel += acc # dt is 1
        pos += vel + signal[i] - prev_sig
        vel *= cs # should be frequency independent
#         out = (pos-signal[i])*np.sqrt(k_spring) # correct for frequency-dependent RMS deviation # LOW
#         out = (pos)*np.sqrt(k_spring) # correct for frequency-dependent RMS deviation # HIGH
        out = (pos-(1-frac_fs)*signal[i])*np.sqrt(k_spring)
        output[i] = out.mean(axis=-1) # average result (volume should be independent of # of frequencies)
        prev_sig = signal[i]

    # return maybe-normalized sum of oscillator values
    return output

In [ ]:
output = wrapper(prototype8, section, ds=0.5, n_f=1000, f_min=10, f_max=10000, tail=2, normalize="raw", seed=0)

In [ ]:
play(output/abs(output).max())

In [ ]:
# NOPE. just doesn't sound good enough. back to the fourier fly wheel.

-----------------------------------

In [ ]:
# general wrapper for pre/post-processing of signal and parameters
def wrapper(
    method, raw_signal, fs=None, ws=None, ts=None,
    f_min=100, f_max=20000, n_f=1000,
    seed=None, sample_rate=44100, tail=2, normalize=False
):
    # convert raw signal to 2d
    raw_signal = np.atleast_2d(raw_signal.T).T

    # initialize input signal with empty buffer tail
    n = len(raw_signal)+tail*sample_rate
    signal = np.zeros((n, raw_signal.shape[1]))
    signal[:len(raw_signal), :] = raw_signal.copy()

    # default frequencies to a random distribution
    if fs is None:
        rng = np.random.default_rng(seed=seed)
        fs = np.exp(np.log(f_min)+(np.log(f_max)-np.log(f_min))*rng.random(n_f)) # log
    elif fs == "linear":
        rng = np.random.default_rng(seed=seed)
        fs = f_min+(f_max-f_min)*rng.random(n_f) # linear

    # set default weights and damping times
    if ws is None:
        ws = np.ones(len(fs))
    if ts is None:
        ts = np.ones(len(fs))*1
    elif isinstance(ts, (float, int)):
        ts = np.ones(len(fs))*ts

    # convert damping (half-life) times to decay coefficients
    cs = np.exp(np.log(0.5)/(ts*sample_rate))

    # normalize weights
    ws /= ws.sum()

    # validate inputs
    assert fs.shape == ws.shape
    assert fs.shape == ts.shape
    assert np.all(cs >= 0) # no negative damping
    assert np.all(cs <= 1) # no amplitude growth

    # retrieve output per channel
    out = np.stack([
        method(channel, sample_rate, fs, ws, cs)
        for channel in signal.T
    ]).T.copy()

    # return optionally-normalized signal
    if normalize is None:
        return np.clip(out, -1, 1)
    elif normalize is False:
        return out / max(1, abs(out).max())
    elif normalize == "raw":
        return out
    else:
        return out / abs(out).max()

In [ ]:
half_time = 0.7

In [ ]:
output = wrapper(prototype3, section[1000:1010], ts=half_time, n_f=1000, f_min=10, f_max=10000, tail=2, normalize="raw", seed=0)

In [ ]:
play(output/abs(output).max())

In [ ]:
# check half-life decay is working
gauss = np.exp(-np.linspace(-5, 5, sample_rate//4)**2)
rms = np.convolve(output[:, 0]**2, gauss)**0.5
plt.figure(figsize=(7, 3))
plt.plot(rms/rms.max())
plt.axhline(rms[40000]/rms.max(), c='k', ls='--')
plt.axhline(rms[40000]/rms.max()/2, c='k', ls='--')
plt.axvline(40000, c='k', ls='--')
plt.axvline(40000+int(sample_rate*half_time), c='k', ls='--')
plt.ylim(0, 0.2)
plt.tight_layout()

-----------------------------------

In [ ]:
# general wrapper for pre/post-processing of signal and parameters
def wrapper(
    method, raw_signal, fs=None, ws=None, ts=None,
    f_min=100, f_max=20000, n_f=1000,
    seed=None, sample_rate=44100, tail=2, normalize=False,
    mono=True,
):
    # convert raw signal to 2d
    raw_signal = np.atleast_2d(raw_signal.T).T

    # initialize input signal with empty buffer tail
    n = len(raw_signal)+tail*sample_rate
    signal = np.zeros((n, raw_signal.shape[1]))
    signal[:len(raw_signal), :] = raw_signal.copy()

    # initialize RNG
    rng = np.random.default_rng(seed=seed)

    # default frequencies to a random distribution
    if fs is None:
        fs = np.exp(np.log(f_min)+(np.log(f_max)-np.log(f_min))*rng.random(n_f)) # log
    elif fs == "linear":
        fs = f_min+(f_max-f_min)*rng.random(n_f) # linear

    # set default weights and damping times
    if ws is None:
        ws = np.ones(len(fs))
    if ts is None:
        ts = np.ones(len(fs))*1
    elif isinstance(ts, (float, int)):
        ts = np.ones(len(fs))*ts

    # convert damping (half-life) times to decay coefficients
    cs = np.exp(np.log(0.5)/(ts*sample_rate))

    # normalize weights
    ws /= ws.sum()

    # validate inputs
    assert fs.shape == ws.shape
    assert fs.shape == ts.shape
    assert np.all(cs >= 0) # no negative damping
    assert np.all(cs <= 1) # no amplitude growth

    # if mono
    if mono:
        # retrieve output per channel
        out = np.stack([
            method(channel[:, None], sample_rate, fs, ws, cs)[:, 0]
            for channel in signal.T
        ]).T.copy()

    # otherwise
    else:
        # run method on signal
        out = method(signal, sample_rate, fs, ws, cs, stereo=True)

    # return optionally-normalized signal
    if normalize is None:
        return np.clip(out, -1, 1)
    elif normalize is False:
        return out / max(1, abs(out).max())
    elif normalize == "raw":
        return out
    else:
        return out / abs(out).max()

In [ ]:
# fourier fly-wheel idea, now with stereo?
def prototype9(signal, sample_rate, fs, ws, cs, stereo=False):
    # set constants
    dt = 1/sample_rate
    os = 2*np.pi*fs*dt
    n, c = signal.shape
    n_f = len(fs)

    # normalize weights
    ws /= ws.sum()

    # validate inputs
    assert fs.shape == ws.shape
    assert fs.shape == cs.shape
    assert np.all(cs >= 0) # no negative damping
    assert np.all(cs <= 1) # no amplitude growth

    #
    mix_LR = np.random.random((n_f,))
    mix_LR = np.stack([mix_LR, 1-mix_LR])

    # initialize containers
    pos = np.zeros((n_f,), dtype=np.complex128)
    output = np.zeros((n, int(stereo)+1))

    # loop through samples
    for i in tqdm.notebook.tqdm(range(n)):
        # unit complex oscillator value
        unit = np.exp(1j*i*os)

        # loop through channels
        for j in range(c):
            # accumulate scaled complex values
            pos += signal[i, j]*unit/c

        # damp cumulant magnitudes
        pos *= cs

        if stereo:
            #
            output[i, :] = (mix_LR*(np.abs(pos)*np.real(unit))).dot(ws)
        else:
            # store oscillator component at cumulant magnitude
            output[i, :] = (np.abs(pos)*np.real(unit)).dot(ws)

    # return output signal
    return output

In [ ]:
section = signal[0:100000, :]

In [ ]:
output = wrapper(prototype9, section[:, 0], ts=0.1, n_f=10, f_min=10, f_max=10000, tail=2, normalize="raw", seed=0, mono=False)

In [ ]:
play(output/abs(output).max())

-----------------------------------

In [ ]:
# let's just do dual mono-based simple prototype 3 resonator

In [ ]:
## full plugin

def process(signal, sample_rate, fs, ws, cs):
    # set constants
    gain = 10000./sample_rate
    os = 2*np.pi*fs/sample_rate
    n = len(signal)
    n_f = len(fs)

    # normalize weights
    ws /= ws.sum()

    # validate inputs
    assert fs.shape == ws.shape
    assert fs.shape == cs.shape
    assert np.all(cs >= 0) # no negative damping
    assert np.all(cs <= 1) # no amplitude growth

    # initialize containers
    pos = np.zeros((n_f,), dtype=np.complex128)
    output = np.zeros((n,))

    # loop through samples
    for i in tqdm.notebook.tqdm(range(n)):
        # unit complex oscillator value
        unit = np.exp(1j*i*os)

        # accumulate scaled complex values
        pos += signal[i]*unit

        # damp cumulant magnitudes
        pos *= cs

        # store oscillator component at cumulant magnitude
        output[i] = (np.abs(pos)*np.real(unit)).dot(ws)*gain

    # return output signal
    return output

def plugin(
    raw_signal, fs=None, ws=None, ts=None,
    f_min=100, f_max=20000, n_f=1000,
    seed=None, sample_rate=44100, tail=2, normalize=False,
):
    # convert raw signal to 2d
    raw_signal = np.atleast_2d(raw_signal.T).T

    # initialize input signal with empty buffer tail
    n = len(raw_signal)+tail*sample_rate
    signal = np.zeros((n, raw_signal.shape[1]))
    signal[:len(raw_signal), :] = raw_signal.copy()

    # initialize RNG
    rng = np.random.default_rng(seed=seed)

    # default frequencies to a random distribution
    if fs is None:
        fs = np.exp(np.log(f_min)+(np.log(f_max)-np.log(f_min))*rng.random(n_f)) # log
    elif fs == "linear":
        fs = f_min+(f_max-f_min)*rng.random(n_f) # linear

    # set default weights and damping times
    if ws is None:
        ws = np.ones(len(fs))
    if ts is None:
        ts = np.ones(len(fs))*1
    elif isinstance(ts, (float, int)):
        ts = np.ones(len(fs))*ts

    # convert damping (half-life) times to decay coefficients
    cs = np.exp(np.log(0.5)/(ts*sample_rate))

    # normalize weights
    ws /= ws.sum()

    # validate inputs
    assert fs.shape == ws.shape
    assert fs.shape == ts.shape
    assert np.all(cs >= 0) # no negative damping
    assert np.all(cs <= 1) # no amplitude growth

    # retrieve output per channel
    out = np.stack([
        process(channel, sample_rate, fs, ws, cs)
        for channel in signal.T
    ]).T.copy()

    # return optionally-normalized signal
    if normalize is None:
        return np.clip(out, -1, 1)
    elif normalize is False:
        return out / max(1, abs(out).max())
    elif normalize == "raw":
        return out
    else:
        return out / abs(out).max()

In [ ]:
# test tones
sample_rate = 44100
test = np.sin(440*2*np.pi*np.arange(2*sample_rate)/sample_rate)
output = plugin(test, ts=0.1, n_f=1, f_min=440, f_max=440, tail=2, normalize="raw", seed=0, sample_rate=sample_rate)
play(output/abs(output).max())
plt.figure(figsize=(6, 2))
plt.plot(test)
plt.plot(output)
plt.tight_layout()

In [ ]:
output = plugin(section, ts=0.1, n_f=100, f_min=100, f_max=10000, tail=2, normalize="raw", seed=0)

In [ ]:
plt.figure(figsize=(6, 3))
plt.plot(section[:, 0])
plt.plot(output[:, 0])
plt.tight_layout()

In [ ]:
# play(section)
# play(output)

-----------------------------------